# Тестовое задание Авито

## Курманаева Эвелина

В данном ноутбуке показан пример работы модели.

* В TEST_DATA можно написать путь к txt файлу, который имеет структуру: id,text_no_spaces, чтобы сформировать submission файл на его подобие.

* Путь к весам модели находится по этой ссылке: https://drive.google.com/file/d/1QUJWzYUeS105B29zNY_aNFHLxd_Z0Lx_/view?usp=share_link

* FILE_ID это id файла из этой ссылки

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import gdown
import pandas as pd

device = (
    "mps"
    if torch.backends.mps.is_available()
    else ("cuda" if torch.cuda.is_available() else "cpu")
)
TEST_DATA = "dataset_1937770_3.txt"
FILE_ID = "1QUJWzYUeS105B29zNY_aNFHLxd_Z0Lx_"
WEIGHT_PATH = 'model_weights.pt'

Cкачиваем модель:

In [2]:
gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", WEIGHT_PATH, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1QUJWzYUeS105B29zNY_aNFHLxd_Z0Lx_
From (redirected): https://drive.google.com/uc?id=1QUJWzYUeS105B29zNY_aNFHLxd_Z0Lx_&confirm=t&uuid=9b2f5a28-1426-416b-9e66-aaaa54f9b0f3
To: /content/model_weights.pt
100%|██████████| 1.11G/1.11G [00:13<00:00, 80.5MB/s]


'model_weights.pt'

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2)
state_dict = torch.load(WEIGHT_PATH, map_location=device)
model.load_state_dict(state_dict)

model.to(device)
model.eval()

Функция 'restore_spaces' для вставки пробелов в предсказанном месте. Пример использования:

In [4]:
def restore_spaces(text):
    encoding = tokenizer(
        text, return_tensors="pt", add_special_tokens=False, return_offsets_mapping=True
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits
        preds = logits.argmax(dim=-1).squeeze().cpu().tolist()

    restored = ""
    for id, p, (start, end) in zip(
        input_ids.squeeze().cpu().tolist(), preds, encoding.offset_mapping[0]
    ):
        tok = tokenizer.decode([id])
        restored += tok
        if p == 1:
            restored += " "
    return restored

In [5]:
example1 = "ищугрузчиковназавтра"
print("INPUT:", example1)
print("OUTPUT:", restore_spaces(example1))

INPUT: ищугрузчиковназавтра
OUTPUT: ищу грузчиков на завтра


In [6]:
example2 = "ищугитариставгруппу"
print("INPUT:", example2)
print("OUTPUT:", restore_spaces(example2))

INPUT: ищугитариставгруппу
OUTPUT: ищу гитаристав группу


In [7]:
example3 = "новаякофеваркаPhilips,гарантияесть"
print("INPUT:", example3)
print("OUTPUT:", restore_spaces(example3))

INPUT: новаякофеваркаPhilips,гарантияесть
OUTPUT: новая кофеварка Philips, гарантия есть


Загрузка тестового датасета и формирование submission файла:

In [8]:
file_path = TEST_DATA
data = []

with open(file_path, "r", encoding="utf-8") as f:
    header = next(f)
    for line in f:
        line = line.strip()
        if not line:
            continue
        id_part, text_part = line.split(",", 1)
        data.append((int(id_part), text_part))


test = pd.DataFrame(data, columns=["id", "text"])


In [9]:
test["normal_sent"] = test["text"].apply(restore_spaces)
test

,id,text,normal_sent
0,0,куплюайфон14про,куплю айфон 14 про
1,1,ищудомвПодмосковье,ищу дом в Подмосковье
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру смебелью и техникой
3,3,новыйдивандоставканедорого,новый диван доставка не дорого
4,4,отдамдаромкошку,отдам даром кошку
...,...,...,...
1000,1000,Янеусну.,Я не у сну.
1001,1001,Весна-яуженегреюпио.,Весна - я ужене грею пио.
1002,1002,Весна-скоровырастеттрава.,Весна - скоро вырастет трава.
1003,1003,"Весна-выпосмотрите,каккрасиво.","Весна - вы посмотрите, как красиво."


Функция для получения индекса, куда вставить пробел

In [10]:
def get_index_space(text):
    i = 0
    indexes = []
    while i < len(text):
        if text[i] == " ":
            indexes.append(i - len(indexes))
            i += 1
        i += 1

    return indexes


In [13]:
test["predicted_positions"] = test.normal_sent.apply(get_index_space)
test

,id,text,normal_sent,predicted_positions
0,0,куплюайфон14про,куплю айфон 14 про,"[5, 10, 12]"
1,1,ищудомвПодмосковье,ищу дом в Подмосковье,"[3, 6, 7]"
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру смебелью и техникой,"[4, 12, 20, 21]"
3,3,новыйдивандоставканедорого,новый диван доставка не дорого,"[5, 10, 18, 20]"
4,4,отдамдаромкошку,отдам даром кошку,"[5, 10]"
...,...,...,...,...
1000,1000,Янеусну.,Я не у сну.,"[1, 3, 4]"
1001,1001,Весна-яуженегреюпио.,Весна - я ужене грею пио.,"[5, 6, 7, 12, 16]"
1002,1002,Весна-скоровырастеттрава.,Весна - скоро вырастет трава.,"[5, 6, 11, 19]"
1003,1003,"Весна-выпосмотрите,каккрасиво.","Весна - вы посмотрите, как красиво.","[5, 6, 8, 19, 22]"


In [14]:
submission = test[["id", "predicted_positions"]]
submission.to_csv("submission.csv", index=False)